## Parse ProQuest Metadata
This notebook includes a python function to parse newspaper articles downloaded from ProQuest Global Newsstream into one CSV file with metadata and full text (when full text is available). 

#### Download PQ files to use as input
The script below takes as input .txt file downloads available via ProQuest Global Newsstream. These are available from ProQuest in batches of up to 100 articles per file. To save those files from your ProQuest search results:
1. Select each article you want to save (or select all results on page) using result checkboxes.
2. From the *...* button dropdown, select "TXT - Text Only"

    ![Screenshot of Saving results from Proquest](imgs/pq_save_feb20.png "Save results from PQ")
3. Accept all defaults and continue to save .txt file of bundled article downloads.
4. Save the downloaded .txt file (or files) to a folder in the same directory as this notebook. In the example below, that folder is called "txt_input" but you can use any path name that you will then call in the final cell.

In [ ]:
## Step 1: import libraries required to run Python code
import os
import re
import glob
import pandas as pd

### Collect metadata
We need to tell the script which fields to collect from the .txt files. You can inspect the text files yourself to look for field names at the beginning of new lines such as `Title: ` or `Publication year: ` and then add them to the list variable called `fieldnames` below. Here is a list of field names available in the ProQuest text downloads as of July 2019:

`'Title', 'Publication title', 'Publication year', 'Document URL', 'Full text', 'Links', 'Section', 'Publication subject', 'ISSN', 'Copyright', 'Abstract', 'Publication info', 'Last updated', 'Place of publication', 'Location', 'Author', 'Publisher', 'Identifier / keyword', 'Source type', 'ProQuest document ID', 'Country of publication', 'Language of publication', 'Publication date', 'Subject', 'Database', 'Document type'`

In [ ]:
fieldnames = ['Title', 'Publication title', 'Publication year', 'Document URL', 'Full text', 'Links', 'Section', 'Publication subject', 'ISSN', 'Copyright', 'Abstract', 'Publication info', 'Last updated', 'Place of publication', 'Location', 'Author', 'Publisher', 'Identifier / keyword', 'Source type', 'ProQuest document ID', 'Country of publication', 'Language of publication', 'Publication date', 'Subject', 'Database', 'Document type']

### Running the script
The function below:
1. Accepts a path to a directory full of .txt files you want to process as its argument (e.g., `parsePQ("txt_input/")`. 
2. Creates a csv file called `pq_metadata.csv`
3. Cycles through every text file in the path you identified in step 1.
4. Splits each document into individual articles using the `sep` separator.
5. Splits each article into lines.
6. For each line, matches `fieldnames` with existing metadata tags in each document. 
7. Saves the fieldname and following content (values) to a dictionary, `metadata_dict`.
7. Writes the fieldnames (keys) and content (values) of the `metadata_dict` for each article to its own row in the CSV.

In [ ]:
sep = "____________________________________________________________"

## function that takes a directory of .txt files from ProQuest as input 
def parsePQ(path, file_output=''): 
    '''This function parses text file downloads from ProQuest into metadata and full-text.
    
    Optional: set the second parameter to 'txt' to also output individual articles as .txt files in /output/
    
    Parameters
    ----------
    
    path : str
        path to .txt files that will be parsed (e.g., 'txts/' or 'pdfs/')
    file_output : str   
        change to file_output='txt' to return individual articles as text files; 
        
    '''
    
    # create a dataframe with fieldname as column headers 
    df = pd.DataFrame(columns = fieldnames)
    
    #cycle through every text file in the directory given as an argument
    files_all = glob.iglob(path + "*.txt")
        
    for filename in files_all:

        #remove the path, whitespace, and '.txt' from filename to later use when printing output
        file_id = filename[:-4].strip(path)

        with open(filename, 'r', encoding='utf-8') as in_file:
            # text var for string of all docs
            text = in_file.read()

            # split string by separator into single articles
            docs = re.split(sep, text)

            # remove first and last items from docs list: first item is empty string; last is copyright info
            docs = docs[1:-1]

            # loop through every doc to collect metadata and full text
            for i, doc in enumerate(docs):
                if file_output == 'txt':
                    new_file = 'output/' + file_id + str(i) + '.txt'
                    txt_file = open(new_file,'w') 
                # remove white space from beginning and end of each article
                doc = doc.strip()

                # skip any empty docs
                if doc=="":
                    continue
                   
                if file_output == 'txt':
                    txt_file.write(doc) 
                    txt_file.close() 
                    
                # split doc on every new line
                metadata_lines = doc.split('\n\n')

                #remove first "line" from article which is the article title without any field title
                metadata_lines = metadata_lines[1:]

                #declare a new dictionary
                metadata_dict = {}

                #for each element add the fieldname/key and following value to a dictionary
                for line in metadata_lines:
                    
                    #ignore lines that do not have a field beginning "Xxxxxx:" (e.g. "Publication title: ")
                    if not re.match(r'^[^:]+: ', line):
                        continue
                    #looks for beginning of new line following structure of "Publication year: " splitting on the colon space
                    (key,value) = line.split(sep=': ', maxsplit=1)
                    
                    #only add to dictionary if the key is in fieldnames
                    if key in fieldnames:
                        metadata_dict[key] = value
                    
                    #look for full text that was cut off by double line breaks and append it to the full text field
                    else:

                        if 'Full text' in metadata_dict.keys():
                            metadata_dict['Full text'] = metadata_dict['Full text'] + line
                
                #write the article to a row in the dataframe
                df.loc[len(df)] = metadata_dict        
            print("Saving", file_id)
    
    #return dataframe 
    return(df)

### Running the script
Running the cell above loads the function, but doesn't do anything yet. 
To *execute* the function, run the cell below, replacing `txts/` with the name of your folder full of .txt files. 

Add a second parameter, file_output='txt', if you would like to return individual articles as text files. In this case make sure there is an ```/output/``` directory available to store the text files.

```df = parsePQ("txt_input/", file_output='txt')```

In [ ]:
#run the function and save to a pandas dataframe
df = parsePQ('txts/')

#remove empty rows
df.dropna(how='all', inplace = True)

#data can be cleaned or examined as a dataframe here


### Save to CSV

In [ ]:
#export to a csv file
df.to_csv('pq_metadata.csv', index=False, encoding='utf-8')

#### Working with the CSV file
Microsoft Excel does not properly render the rows for some articles with unique line breaks. If you open the CSV file and find that the metadata for some articles is breaking up improperly over columns, you can follow these directions to work with the dataset:
1. Create a new blank Google Sheet and choose File > Import. 
2. Select the CSV file that you exported and in the pop up change the Separator type to "Comma" and uncheck the "Convert text..." option. The data should be properly parsed in Google Sheets. 
![Pop-up box offering how to import the CSV file](imgs/import_file.png "Import file into Google Sheets")
3. If you prefer to work with the data in Excel, then go to File > Download as > Microsoft Excel and save a copy to your computer.
4. When opening the file you may get an alert from Excel. If so, choose "Yes" to recover data.

![Excel alert warning there may be problems with the metadata](imgs/alert.png "Excel alert")

Created by Cody Hennesy and David Naughton (University of Minnesota, Twin Cities, Libraries). Feel free to email Cody (chennesy@umn.edu) with any questions. 